In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
file = '../data/NASA Near-Earth Objects-CleanbyThang.csv'
df = pd.read_csv(file,index_col=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23999 entries, 2001981 to 54073367
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   absolute_magnitude_h               23999 non-null  float64
 1   is_potentially_hazardous_asteroid  23999 non-null  bool   
 2   is_sentry_object                   23999 non-null  bool   
 3   kilometers_estimated_diameter_min  23999 non-null  float64
 4   kilometers_estimated_diameter_max  23999 non-null  float64
 5   orbit_class_type                   23999 non-null  object 
 6   perihelion_distance                23999 non-null  float64
 7   aphelion_distance                  23999 non-null  float64
 8   first_observation_date             23999 non-null  object 
 9   last_observation_date              23999 non-null  object 
 10  orbit_class_description            23999 non-null  object 
 11  first_observation_year             23999 non-null 

In [4]:
df['first_observation_date'] = pd.to_datetime(df['first_observation_date'])
df['last_observation_date'] = pd.to_datetime(df['last_observation_date'])



In [5]:
df= df.drop(columns=['orbit_class_description'])

In [11]:
# đánh giá ma trận tương quan

category_col = ['orbit_class_type','is_sentry_object','is_collidable']
numeric_col = ['absolute_magnitude_h','kilometers_estimated_diameter_min','kilometers_estimated_diameter_max','perihelion_distance','aphelion_distance','first_observation_year','last_observation_year']
target_col = 'is_potentially_hazardous_asteroid'


In [14]:
# để đánh giá tương quan một biến dàng nhị phân và các biến liên tục ta dùng Point Biserial Correlation
from scipy.stats import pointbiserialr
corr_numeric = []
for col in numeric_col:
    if df[col].dtype == 'datetime64[ns]':
        df[col] = df[col].dt.year
    corr = pointbiserialr(df[target_col],df[col])
    corr_numeric.append([col,corr[0],corr[1]])
    
for res in corr_numeric:
    if res[2] < 0.05:
        print(f"{res[0]}: {res[2]}")
    else:
        print(f"{res[0]}: {res[2]} - Không có tương quan")

absolute_magnitude_h: 0.0
kilometers_estimated_diameter_min: 1.9584104390080013e-164
kilometers_estimated_diameter_max: 1.9584104376160863e-164
perihelion_distance: 3.1526025182785987e-229
aphelion_distance: 0.1234479157224278 - Không có tương quan
first_observation_year: 1.5763385521265077e-219
last_observation_year: 2.2150925877600762e-55


In [13]:
# anova 
from scipy.stats import f_oneway
anova = []
for col in category_col:
    group = df.groupby(col).groups
    group = [df.loc[group[i],target_col] for i in group]
    anova.append([col,f_oneway(*group)])
for res in anova :
    if res[1][1] < 0.05:
        print(f"{res[0]}: {res[1][0]} p={res[1][1]}")
    else:
        print(f"{res[0]}: {res[1][0]} p={res[1][1]} - Không có tương quan")

orbit_class_type: 352.4678834036408 p=5.107238872039364e-224
is_sentry_object: 83.18920211588798 p=8.026294502238391e-20
is_collidable: 1004.2752234054757 p=5.938465832700636e-216


chọn các cột sau 
- cate_train = ['orbit_class_type','is_sentry_object','is_collidable']
- numeric_train = ['absolute_magnitude_h','kilometers_estimated_diameter_min','kilometers_estimated_diameter_max','perihelion_distance','first_observation_year','last_observation_year']